## Szegmentálás és tokenizálás tesztelni egy példa fasta file-on, ami a ESKAPE pathogenekből szárzmazik, ez lehet a példa kód a későbbi feladatokra.


In [19]:
%load_ext autoreload
%autoreload 2
from transformers import AutoTokenizer, AutoModel, utils
#from bertviz import model_view
utils.logging.set_verbosity_error()  # Suppress standard warnings
import yaml
import pathlib
from os.path import join
import os
import sys
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

current_path = str(pathlib.Path(os.getcwd()).parent)
sys.path.append(current_path)

#import sys
#sys.path.append('../)

from config_utils import *
from sequtils import *

%load_ext autoreload
%autoreload 2

defconfig = SeqConfig()
segmentation_params = defconfig.get_set_segmentation_parameters()
tokenization_params = defconfig.get_and_set_tokenization_params()             
comp_params = defconfig.get_set_computational_paramters()
samplefasta_file = '/home/ligeti/github/prokbert/src/prokbert/data/sample_data/ESKAPE_sample.fasta'
contigs = load_contigs(samplefasta_file,IsAddHeader=True,AsDataFrame=True, adding_reverse_complement=False)
contigs['sequence_id'] = [0, 1, 2]
sequences_ls = list(contigs['sequence'])

act_segments = segment_sequence_contiguous(sequences_ls[0], segmentation_params, 1)
segmentation_params['type'] = 'random'
results = segment_sequences(contigs, segmentation_params, AsDataFrame=True)


2023-08-10 13:45:36,326 - INFO - Loading sequence data into memory!
2023-08-10 13:45:36,327 - INFO - Since the fasta_files_list is a string, not list, we convert to a list.
2023-08-10 13:45:36,329 - INFO - Checking input DataFrame!
2023-08-10 13:45:36,329 - INFO - Checking input sequence_id is valid primary key in the DataFrame
2023-08-10 13:45:36,330 - INFO - Sampling 19 segments from 3 sequences.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
SEQ_CONFIG_FILE environment variable has not been set. Using default value: /home/ligeti/github/prokbert/src/prokbert/configs/sequence_processing.yaml
/home/ligeti/github/prokbert/src/prokbert
/home/ligeti/github/prokbert/src/prokbert


{'cpu_cores_for_segmentation': 10,
 'cpu_cores_for_tokenization': 48,
 'batch_size_tokenization': 10000,
 'batch_size_fasta_segmentation': 3}

In [25]:
segments = list(results['segment'])
segment_ids = list(results['segment_id'])

num_cores = comp_params['cpu_cores_for_tokenization']
batch_size = comp_params['batch_size_tokenization']

r = batch_tokenize_segments_with_ids([segments, segment_ids], tokenization_params, num_cores, batch_size)
r = batch_tokenize_segments_with_ids(results, tokenization_params, num_cores, batch_size)


2023-08-10 13:57:31,217 - INFO - Tokenization of a list of segments
2023-08-10 13:57:31,426 - INFO - Tokenization of a list of segments


In [ ]:
n_sequence


In [ ]:
n_sequence = [ str[0:50] for str in contigs['sequence']]
test_contigs =contigs [ ['sequence_id', 'sequence']]
test_contigs['sequence'] = n_sequence
test_contigs.to_json()


In [ ]:
segment = 'TCTTTGCTAAG'
params_example = {
            'shift': 1, 
            'max_segment_length': 512, 
            'max_unknown_token_proportion': 0.2, 
            'kmer': 5, 
            'token_limit': 10
        }
params_example = defconfig.get_and_set_tokenization_params(params_example)         

result = lca_tokenize_segment(segment, params_example)

print(result)

In [ ]:
segment_sequences(contigs, segmentation_params, AsDataFrame = True)
 


## Adding the coverage based segmentation


1. lépés valamilyen dataframe-t létrehozni, amiben
cumsum, contig_lengths, sequences, contigs, ilyesmi. De nem tudom ezek között mi is a különbség



In [ ]:
params = segmentation_params
params['type'] = 'random'
results = segment_sequences(contigs, params, AsDataFrame=True)
results = segment_sequences(contigs, params, AsDataFrame=False)

#pd.DataFrame(results)
results = segment_sequences(contigs, params, AsDataFrame=True)


# Tokenizáló tesztelése


In [ ]:
tokenization_params = defconfig.get_and_set_tokenization_params()         
tokenization_params['shift']=2
tokenization_params['kmer']=5
print(list(results['segment'])[0][0:30])

tokens, kmers = lca_tokenize_segment(list(results['segment'])[0][0:30], tokenization_params)
#tokens, kmers = lca_tokenize_segment('ATCTGT', tokenization_params)
print(tokens)
print(kmers)
#tokens
#print(kmers)


In [ ]:
tokenization_params['vocabmap']['[CLS]']

In [ ]:
import json
# Specify the file path
file_path = "/home/ligeti/tokenizer_params.json"

# Write the dictionary to a JSON file
with open(file_path, "w") as json_file:
    json.dump(tokenization_params, json_file, indent=4)  # indent argument adds pretty formatting



In [ ]:
tokenization_params = defconfig.get_and_set_tokenization_params()
segments = list(results['segment'])
segment_ids = list(results['segment_id'])
batch_tokenize_segments_with_ids(segments, segment_ids, tokenization_params)
